In [ ]:
import os
import h5py
import stripy
import meshplex
import numpy as np
import pyvista as pv
from pathlib import Path
from scipy import interpolate
from PlateBoundaries import *
from TectonicEarth import Earth
from gospl.model import Model as sim
from gospl._fortran import definegtin
from EarthsAssistant import EarthAssist

In [ ]:
#Some code to check that I haven't broken Earth each whenever I edit it
if False:
    earth = Earth(
                startTime = 20,
                endTime = 0,
                deltaTime = 1,
                baseUplift = 2000,
                distTransRange = 1000000, 
                numToAverageOver = 10,
                earthRadius = 6378137.,
                useKilometres = False
    )

    earth.runTectonicSimulation()
    earth.animate(lookAtLonLat=[60, 20], cameraZoom=1.4)

# Gospl

Gospl is a python based numerical model for simulating various global scale erosion processes. It takes plate tectonic landscape displacements along with climatic data as input, and simulates *long-term catchment dynamic and drainage evolution as well as sedimentary basins formation*. We will analyze how to run simulations in Gospl to gain a better understanding of it.

### Input yml Files

In Gospl, simulation properties and input files are specified by *.yml* files. We will discuss an example input *.yml* file, and provide examples for generating and reading data referenced by the *.yml* files. For a more detailed documentation of *.yml* attributes used in *Gospl*, see the [official user guide](https://gospl.readthedocs.io/en/latest/user_guide/inputfile.html) documentation. Note that some sentences in this notebook were copied and pasted from the user guide.

##### Domain

A *Gospl* input *yml* file begins by specifying the name, and the domain of the erosion simulation. It declares the location of the *.npz* file containing data of the initial surface mesh to be used by *Gospl*, some other simulation parameters, and takes the following form:

``` yml
name: Example input yml file

domain:
  npdata: 'ExampleInput/elev15Ma'
  flowdir: 5
  fast: False
  backward: False
  interp: 1
 ```
 
The main parameter of our interest here is *npdata*, which specifies the file location of the initial surface mesh to be used by Gospl. This file needs to be of *.npz* type, which stores numpy zipped arrays. To identify the structure of the arrays that these files need to take, we will inspect *ExampleInput/elev15Ma.npz*, which was taken from the Gospl *bfModel* notebook tutorial, and contains the following arrays:

- v: Vertices
- c: Cells
- n: Nearest Neighbours
- z: Elevations

In [ ]:
#Load npz file
npzFile = np.load('ExampleInput/elev15Ma.npz')
print(npzFile.files)

#Get arrays stored in file
vertices = npzFile['v']
cells = npzFile['c']
nearestNeighbours = npzFile['n']
elevations = npzFile['z']

#Print the shape of arrays within the npz file
print(vertices.shape)
print(cells.shape)
print(nearestNeighbours.shape)
print(elevations.shape)

We can create a pyvista mesh object for visualizations of the earth's initial elevation from the npz file. To do so we will require an array of the exagerated earth's radius, and a list of faces, which can be done as follows:

In [ ]:
#Read npz file and return numpy arrays
def readNPZelevationFile(npzFileDir):
    npzFile = np.load(npzFileDir)
    elevation = npzFile['z']
    if len(elevation.shape) == 1:
        elevation = elevation[:, np.newaxis]
    return npzFile['v'], npzFile['c'], npzFile['n'], elevation

#Given the data from the NPZ file, we create an pyvista mesh of earth
def createMeshFromNPZdata(vertices, cells, heights, heightAmplification=30):
    faces = []
    exageratedRadius = vertices + heightAmplification * heights * vertices / np.max(vertices)
    for cell in cells:
        faces.append(3)
        faces.append(cell[0])
        faces.append(cell[1])
        faces.append(cell[2])
    earthMesh = pv.PolyData(exageratedRadius, faces)
    earthMesh['heights'] = heights
    return earthMesh

#Run the newly created functions
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev15Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

To combine our Tectonic Earth with Gospl simulations, we need to create a similar *.npz* file to initialize Gospl with. Because erosion algorithms used in Gospl depend on nearby neighbours of vertices, it benefits from using an icosahedral sphere (Icosphere), instead of the UV sphere that our earth has been based on so far. We begin by interpolating heights from our UV sphere to heights on an Icosphere, where the Icosphere is generated by the stripy library. The function bellow takes an *Earth* object as input, and returns arrays of the vertices, cells and elevations for our *.npz* file.

In [ ]:
#Create an icosphere
def createIcosphere(subdivisions=6, radius=6378137):
    icosphere = stripy.spherical_meshes.icosahedral_mesh( 
                    refinement_levels = subdivisions,
                    include_face_points = False)
    icosphereXYZ = icosphere._points * radius
    icoCells = icosphere.simplices
    return icosphereXYZ, icoCells

#Create an icosphere and interpolate earth heights onto it
def earthToIcosphere(earth, subdivisions=6, iteration=-1):
    icosphereXYZ, icoCells = createIcosphere(subdivisions=subdivisions, radius=earth.earthRadius)
    
    #Interpolate heights
    earthHeights = earth.heightHistory[iteration]
    radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoHeights = interpolate.griddata(earth.lonLat, earthHeights, icoLonLat, method='cubic')
    icoHeights = icoHeights[:, np.newaxis]
    return icosphereXYZ, icoCells, icoHeights

The last required array is that of the nearest neighbours IDs, which we create as shown bellow, where the code is based on the *bfModel* notebook tutorial. We then define a function for writing the file, and to confirm that it is written correctly, we open and display it.

In [ ]:
#Create list of neighbour ids based on bfModel notebook tutorial
def getNeighbourIds(icoXYZ, icoCells):
    Gmesh = meshplex.MeshTri(icoXYZ, icoCells)
    s = Gmesh.idx_hierarchy.shape
    a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
    Gmesh.edges = {'points': np.unique(a, axis=0)}
    ngbNbs, ngbID = definegtin(len(icoXYZ), Gmesh.cells['points'], Gmesh.edges['points'])
    ngbIDs = ngbID[:,:8].astype(int)
    return ngbIDs

#Create an appropriate npz file from earth object and return the file name of the new npz file
def createNPZfromEarth(earth, outDirectory='GeneratedInputFiles/', iteration=-1, subdivisions=6):
    vertices, cells, heights = earthToIcosphere(earth, subdivisions=subdivisions)
    neighbours = getNeighbourIds(vertices, cells)
    
    #Create appropriate file name and save data as npz
    time = earth.timeHistory[iteration]
    fileName = '{}ElevationSubdivisions{}Time{}Mya'.format(outDirectory, subdivisions, time)
    np.savez_compressed(fileName, v=vertices, c=cells, n=neighbours.astype(int), z=heights)
    return fileName + '.npz'
    

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 10,
            endTime = 0,
            deltaTime = 2,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Create the npz file to be used by Gospl
elevationsFileName = createNPZfromEarth(earth)

#Read the newly created npz file as before to check that everything is working as expected
vertices, cells, neighbours, heights = readNPZelevationFile(elevationsFileName)
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

##### Time

Now that we have discussed the domain of the gospl simulation and the required input files, the next set of parameters that need to be specified in the *input.yml* file for Gospl is time, which has the following format:

``` yml
time:
  start: -10000000.
  end: 0.
  tout: 2000000.
  dt: 200000.
  tec: 2000000.
 ```
According to the [Gospl user documentation](https://gospl.readthedocs.io/en/latest/user_guide/inputfile.html), these parameters are described as:


- `start` is the model start time in years,

- `end` is the model end time in years,

- `tout` is the output interval used to create model outputs,

- `dt` is the model internal time step (the approach in *gospl* uses an implicit time step).

- `tec` is the tectonic timestep interval used to update the tectonic meshes and perform the required displacements.

##### Stream Power Law (SPL)

The Stream Power Law is a family of equations that governs erosion processes due to water flow (Eg. Rivers and rainfall). The defaults values are set as follows:

``` yml
spl:
  wfill: 100.
  sfill: 10.
  K: 3.e-8
  d: 0.42
```

##### Diffusion

Hillslope processes in gospl is defined using a classical diffusion law in which sediment deposition and erosion depend on slopes. The default parameters are as follows:

``` yml
diffusion:
  hillslopeKa: 0.02
  hillslopeKm: 0.2
  dstep: 5
  sedK: 100.
  sedKf: 200.
  sedKw: 300
```

##### Sea Level Forcing

Here we specify the sea level for the simulation. The *curve* parameter is optional and specifies the sea level over time, and if it is excluded from the *yml* file, then a constant default position of 0 will be used throughout the simulation.

``` yml
sea:
  position: 0.
  curve: 'data/sealevel.csv'
```

##### Tectonic

This section specifies a series tectonic forcing parameters, and can be specified as follows:

``` yml
tectonic:
  - start: -15000000.
    end: -14000000.
    mapH: 'input8/disp15Ma'
  - start: -14000000.
    end: -13000000.
    mapH: 'input8/disp14Ma'
  - start: -13000000.
    end: -12000000.
    mapH: 'input8/disp13Ma'
    ...
    ...
    ...
  - start: -1000000.
    end: 0.
    mapH: 'input8/disp1Ma'
```

Here the tectonic uplift force is specified by multiple *.npz* file at various times. Again, we will analyze these files and show how we can generate them from our tectonic earth.

These files contain *XYZ* coordinates of the tectonic forces during the specified time period. These arrays need to be of shape (M, 3), where M is the number of vertices on our Icosphere. To visualize these files, we will use *pyvista* glyphs to create a vector field, where an arrow has been placed just above earth to depict the direction and magnitude of the force. Instead of placing an arrow for each vertex on the sphere, we will reduce the vertices to a subset where we only use every N vertices.

In [ ]:
def readNPZtectonicFile(npzFileDir):
    npzFile = np.load(npzFileDir)
    print(npzFile['xyz'].shape)
    return npzFile['xyz']

def createTectonicVectorMesh(vertices, displacementXYZ, ignoreEveryNverts=50, vectorScale=600000):
    dispMagnitude = np.linalg.norm(displacementXYZ, axis=1)
    dispDirection = displacementXYZ / dispMagnitude[:, np.newaxis]
    dispMagnitude /= np.max(dispMagnitude)
    
    arrow = pv.Arrow()
    reduceIndex = (np.arange(vertices.shape[0]) % ignoreEveryNverts == 0)
    vectorLocations = pv.PolyData(vertices[reduceIndex] * 1.02)
    vectorLocations['dispMag'] = dispMagnitude[reduceIndex]
    vectorLocations['dispDir'] = dispDirection[reduceIndex]
    forceVectors = vectorLocations.glyph(orient="dispDir", scale="dispMag", factor=vectorScale, geom=arrow)
    return forceVectors

displacementXYZ = readNPZtectonicFile('ExampleInput/disp15Ma.npz')
vertices, cells, neighbours, heights = readNPZelevationFile('ExampleInput/elev15Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
forceVectors = createTectonicVectorMesh(vertices, displacementXYZ)

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(forceVectors)
plotter.add_mesh(earthMesh.contour([0]), color="black", opacity=1.)
plotter.show()

We now need to use our Earth script to generate an *.npz* file similar to the one above.

In [ ]:
def getTectonicForce(earth, iteration=-1, maxTectonicForce=0.18):
    earthBeforeXYZ = earth.getEarthXYZ(amplifier=1, iteration=iteration-1)
    #earthAfterXYZ = earth.getEarthXYZ(amplifier=1, iteration=iteration)
    
    heightsAfter = earth.heights
    radius = heightsAfter + earth.earthRadius
    earthAfterXYZ = EarthAssist.polarToCartesian(radius, earth.movedLonLat[:, 0], earth.movedLonLat[:, 1])
    
    forceXYZ = (earthAfterXYZ - earthBeforeXYZ)
    forceXYZ *= maxTectonicForce / np.max(np.linalg.norm(forceXYZ, axis=1))
    return forceXYZ

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 20,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

earth.doSimulationStep(earth.startTime)
#earth.doSimulationStep(earth.startTime - earth.deltaTime)
#earth.doSimulationStep(earth.startTime - 2 * earth.deltaTime)

forceXYZ = getTectonicForce(earth)
earthXYZ = earth.getEarthXYZ(amplifier=1, iteration=-1)
forceVectors = createTectonicVectorMesh(earthXYZ, forceXYZ, ignoreEveryNverts=2, vectorScale=200000)

#Display the results
plotter = pv.PlotterITK()
earthMesh = earth.getEarthMesh(iteration=-1)
plotter.add_mesh(earthMesh, scalars=earth.heightHistory[-1])
plotter.add_mesh(forceVectors)
plotter.show()

In [ ]:
#We interpolate the force field onto an Icosphere suitable for Gospl
def interpolateForces(earth, icosphereXYZ, forceXYZ):
    radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoForce = interpolate.griddata(earth.lonLat, forceXYZ, icoLonLat, method='cubic')
    return icoForce

#And create a function for writing force field files at the latest tectonic earth simulation step
def createForceNPZfile(earth, icosphereXYZ, outDirectory='GeneratedInputFiles/', subdivisions=6):
    forceXYZ = getTectonicForce(earth)
    icoForce = interpolateForces(earth, icosphereXYZ, forceXYZ)
    
    #Need to create file
    time = earth.timeHistory[-2]
    fileName = '{}ForceSubdivisions{}Time{}Mya'.format(outDirectory, subdivisions, time)
    np.savez_compressed(fileName, xyz=icoForce)
    return fileName + '.npz'

In [ ]:
#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 20,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Create the elevation file to be used by Gospl
elevationsFileName = createNPZfromEarth(earth)
icosphereXYZ, icoCells, icoNeighbs, icoHeights = readNPZelevationFile(elevationsFileName)

#The main loop in earth.runTectonicSimulation() will now look something like this
fileNames = []
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    
    earth.doSimulationStep(time)
    forceFileName = createForceNPZfile(earth, icosphereXYZ)
    fileNames.append(forceFileName)

In [ ]:
#Create meshes for visualising the newly created files
displacementXYZ = readNPZtectonicFile(fileNames[0])
vertices, cells, neighbours, heights = readNPZelevationFile(elevationsFileName)
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
forceVectors = createTectonicVectorMesh(vertices, displacementXYZ, ignoreEveryNverts=30)

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(forceVectors)
plotter.add_mesh(earthMesh.contour([0]), color="black", opacity=1.)
plotter.show()

Instead of manually writing out this *.yml* specification, we automatically generate it here so that we can copy and paste it.

In [ ]:
textFormat = " - start: -{}.\n   end: -{}.\n   mapH: '{}'"
for i, time in enumerate(earth.simulationTimes[:-1]):
    print(textFormat.format(1000000 * time, 1000000 * (time - earth.deltaTime), (fileNames[i])[:-4]))

##### Climate

The climate specifies the amount of rainfall at particular locations of the earth. Although Gospl allows us to specify it using another *.npz* file, we can also specify it to be uniformly distributed across the entire earth, which is done as follows.

``` yml
climate:
  - start: -15000000.
    uniform: 1.
```

##### Output

Finally, we specify the output directory for the file generated by Gospl.

``` yml
output:
  dir: 'GosplOutput'
  makedir: False
```

# Automatic Creation of YML files

Since the process of writing *.yml* files and generating their corresponding *.npz* files is tedious, we will automate this process by defining a Gospl manager class.

In [ ]:
class GosplManager:
    def __init__(self, earth,
                dataDirectory = 'GeneratedInputFiles',
                subdivisions=6
                ):
        
        self.earth = earth
        self.dataDirectory = dataDirectory
        self.subdivisions = subdivisions
        self.mainOutputDirectory = 'GosplRuns'
        
        #============================== Default Attributes ======================================================
        #Name attribute
        self.name = 'Automatically generated YML file'
        
        #Domain attributes
        #self.npdata = dataDirectory + '/Elevations{}Mya'.format(subdivisions, earth.startTime)
        self.npdataFormat = '{}/Elevations{}Mya'
        self.flowdir = 5
        self.fast = False
        self.backward = False
        self.interp = 1
        
        #Time attributes
        self.start = earth.startTime * 1000000
        self.end = earth.endTime * 1000000
        self.tout = earth.deltaTime * 1000000
        self.dt = earth.deltaTime * 1000000 / 5
        self.tec = earth.deltaTime * 1000000
        
        #Stream Power Law attributes (SPL)
        self.wfill = 100.
        self.sfill = 10.
        self.K = 3e-8
        self.d = 0.42
        
        #Diffusion attributes
        self.hillslopeKa = 0.02
        self.hillslopeKm = 0.2
        self.dstep = 5
        self.sedK = 100.
        self.sedKf = 200.
        self.sedKw = 300
        
        #Sea level attributes
        self.position = 0.
        
        #Climate attributes
        self.uniform = 1.
        
        #Output Attributes
        self.dirFormat = 'GosplOutput{}'
        self.makedir = False
    
    #============================== YML File Creation ======================================================
    #We generate a string containing all the content of the YML file
    #For each section of the YML file, we have a seperate function creating a string for it,
    #We then combine all these sections into a single string and create the YML file
    def getNameString(self):
        return "\nname: {}\n\n".format(self.name)
    
    def getDomainString(self):
        domainFormat = "domain:\n  npdata: '{}'\n  flowdir: {}\n  fast: {}\n  backward: {}\n  interp: {}\n\n"
        npdata = self.npdataFormat.format(self.npzFilesDirectory, earth.startTime)
        domainString = domainFormat.format(
            npdata,
            self.flowdir,
            self.fast,
            self.backward,
            self.interp)
        return domainString
    
    def getTimeString(self):
        timeFormat = "time:\n  start: -{}\n  end: {}\n  tout: {}\n  dt: {}\n  tec: {}\n\n"
        timeString = timeFormat.format(
            float(self.start),
            float(self.end),
            float(self.tout),
            float(self.dt),
            float(self.tec))
        return timeString
    
    def getSPLstring(self):
        splFormat = "spl:\n  wfill: {}\n  sfill: {}\n  K: {}\n  d: {}\n\n"
        splString = splFormat.format(
            self.wfill,
            self.sfill,
            self.getYMLscientificNotation(self.K),
            self.d)
        return splString
    
    def getDiffusionString(self):
        diffusionFormat = "diffusion:\n  hillslopeKa: {}\n  hillslopeKm: {}\n  dstep: {}\n  sedK: {}.\n  sedKf: {}.\n  sedKw: {}\n\n"
        diffusionString = diffusionFormat.format(
            self.hillslopeKa,
            self.hillslopeKm,
            self.dstep,
            int(self.sedK),
            int(self.sedKf),
            self.sedKw)
        return diffusionString
    
    def getSeaString(self):
        seaFormat = "sea:\n  position: {}\n\n"
        seaString = seaFormat.format(self.position)
        return seaString
    
    def getTectonicString(self):
        tectonicString = "tectonic:\n"
        tectonicPartFormat = " - start: -{}.\n   end: -{}.\n   mapH: '{}/ForceSubdivisions{}Time{}Mya'\n"
        for time in self.earth.simulationTimes[:-1]:
            tectonicPart = tectonicPartFormat.format(
                1000000 * time, 
                1000000 * (time - earth.deltaTime),
                self.dataDirectory,
                self.subdivisions,
                time)
            tectonicString += tectonicPart
        return tectonicString + '\n'
    
    def getClimateString(self):
        climateFormat = "climate:\n  - start: -{}.\n    uniform: {}\n\n"
        climateString = climateFormat.format(
            1000000 * self.earth.startTime,
            self.uniform)
        return climateString
    
    def getOutputString(self):
        index = 1
        outputFormat = "output:\n  dir: '{}'\n  makedir: {}\n\n"
        while os.path.isdir(self.dirFormat.format(index)):
            index += 1
        dirName = self.dirFormat.format(index)
        outputString = outputFormat.format(
            dirName,
            self.makedir)
        return outputString
    
    #Create a string containing all the content of the YML file
    def getYMLstring(self):
        name = self.getNameString()
        domain = self.getDomainString()
        time = self.getTimeString()
        spl = self.getSPLstring()
        diffusion = self.getDiffusionString()
        sea = self.getSeaString()
        tectonic = self.getTectonicString()
        climate = self.getClimateString()
        output = self.getOutputString()
        return name + domain + time + spl + diffusion + sea + tectonic + climate + output
    
    #If x is given in scientific notation, return x as string scientific notation compatible with YML files.
    @staticmethod
    def getYMLscientificNotation(x):
        if 'e' in str(x):
            x = str(x).split('e-')
            x = '{}.e-{}'.format(x[0], x[1])
        return x
    
    def makeFiles(self):
        
        #Create the main output directory if it does not already exist
        if not os.path.isdir(self.mainOutputDirectory):
            os.mkdir('./{}'.format(self.mainOutputDirectory))
        
        #Create subdirectory for this particular Gospl run
        runNumber = 1
        thisRunDirectory = './{}/run{}'.format(self.mainOutputDirectory, runNumber)
        while os.path.isdir(thisRunDirectory):
            runNumber += 1
            thisRunDirectory = './{}/run{}'.format(self.mainOutputDirectory, runNumber)
        os.mkdir(thisRunDirectory)
        
        #Create directory for npz files
        self.npzFilesDirectory = thisRunDirectory + '/NPZfiles'
        os.mkdir(self.npzFilesDirectory)
        
        #Create the forward YML file
        ymlFileContent = self.getYMLstring()
        ymlFileDirectory = thisRunDirectory + '/forward.yml'
        ymlFile = open(ymlFileDirectory, 'w')
        ymlFile.write(ymlFileContent)
        ymlFile.close()

gosplMan = GosplManager(earth)
gosplMan.makeFiles()
            

In [ ]:
x = 3e-08
#x = 10
if 'e' in str(x):
    x = str(x).split('e-')
    x = '{}.e-{}'.format(x[0], x[1])
print(x)


# Running A Gospl Simulation

We run the Gospl simulation as follows

In [ ]:
if False:
    inputYMLfile = 'input.yml'
    mod = sim(inputYMLfile, False, False)
    mod.runProcesses()
    mod.destroy()

After running a Gospl simulation, a folder named 'h5' should be generated within the output directory specified by the *.yml* file. This folder contains the output data generated by Gospl. It contains a one *topology.p0.h5* file with mesh data of the earth, and a series of files with the naming scheme *gospl.{}.p0.h5* containing simulation output data at each time step *tout*, where *tout* was specified by the *.yml* file, and the symbol *{}* in the naming scheme represents the simulation iteration.

Each file of the format *gospl.{}.p0.h5* contains the following data for each vertex on the earth's sphere:

- `elev`: The height of the vertex

- `erodep`: The amount of soil deposited at this vertex

- `fillAcc`: The amount of water on a vertex

- `flowAcc`: The flow of water on a vertex

- `rain`: The amount of rainfall on a given vertex

- `sedLoad`: The amount of sediment being carried by the water 

Note that the above attributes are not documented in the Gospl documentations and are based on my best guess. (Note to Tristan: You might want to add these to your user documentations).

We now provide an example of how to visualize the gospl output data files. Note that for the purpose of visualizations, we have raised some attributes by the power of 0.25 to make features more visible. To specify the attribute to visualize, use the dropdown menu located in the top left of the render window, alternatively we can change the 'scalars' argument in the *plotter.add_mesh()* function call.

In [ ]:
#The Gospl file contains simulation output data at particular iterations during the simulation
def readGosplFile(fileDir):
    gosplDict = {}
    with h5py.File(fileDir, "r") as f:
        for key in f.keys():
            gosplDict[key] = np.array(f[key])
    return gosplDict

#Using the gospl data, we create a pyvista mesh
def createGosplDataMesh(gosplFilenamePattern, iteration, radius=6378137, heightAmplification=30, subdivisions=8):
    gosplData = readGosplFile(gosplFilenamePattern.format(iteration))
    icoXYZ, icoCells = createIcosphere(subdivisions=subdivisions, radius=earth.earthRadius)
    outputMesh = createMeshFromNPZdata(icoXYZ, icoCells, gosplData['elev'], heightAmplification=heightAmplification)
    
    #Store gospl data as mesh attributes
    outputMesh['elev'] = gosplData['elev'] 
    outputMesh['erodep'] = gosplData['erodep'] 
    outputMesh['fillAcc'] = gosplData['fillAcc']**0.25
    outputMesh['flowAcc'] = gosplData['flowAcc']**0.25
    outputMesh['rain'] = gosplData['rain'] 
    outputMesh['sedLoad'] = gosplData['sedLoad']**0.25
    return outputMesh

#Specify the data file to visualize and create a mesh from it
iteration = 2
outputDir = 'GosplOutput2/h5/'
gosplFilenamePattern = outputDir + 'gospl.{}.p0.h5'
outputMesh = createGosplDataMesh(gosplFilenamePattern, iteration)

#Plot the results
plotter = pv.PlotterITK()
plotter.add_mesh(outputMesh, scalars='flowAcc')
plotter.show()

In [ ]:
#Animate the results produced by Gospl
def animateGosplOutput(outputDir, subdivisions,
                       scalarAttribute = 'elev',
                       gosplFilenamePattern = 'gospl.{}.p0.h5',
                       movieOutputFileName='GosplAnimation.mp4',
                       lookAtLonLat = np.array([60, 20]),
                       cameraZoom = 1.4,
                       framesPerIteration = 8):
    
    #Set up directories, get number of file for animation, and initialise the animation mesh
    outputPath = Path(outputDir)
    numOfFiles = len(list(outputPath.glob('gospl.*.p0.h5')))
    fileNamePattern = outputDir + gosplFilenamePattern
    earthMesh = createGosplDataMesh(fileNamePattern, 0, subdivisions=subdivisions)
    
    #Set up plotter object and camera position for animation
    plotter = pv.Plotter()
    plotter.add_mesh(earthMesh, scalars=scalarAttribute)
    plotter.camera_position = 'yz'
    plotter.camera.zoom(cameraZoom)
    plotter.camera.azimuth = 180 + lookAtLonLat[0]
    plotter.camera.elevation = lookAtLonLat[1]
    plotter.show(auto_close=False, window_size=[800, 608])
    plotter.open_movie(movieOutputFileName)
    for i in range(framesPerIteration):
        plotter.write_frame()
    
    #Iterate through files and write animation frames
    for i in range(numOfFiles-1):
        newMesh = createGosplDataMesh(fileNamePattern, i+1, subdivisions=subdivisions)
        plotter.update_coordinates(newMesh.points, mesh=earthMesh)
        plotter.update_scalars(newMesh[scalarAttribute], render=False, mesh=earthMesh)
        for i in range(framesPerIteration):
            plotter.write_frame()
    plotter.close()
    return

animateGosplOutput('./GosplOutput/h5/', 6)
#animateGosplOutput('./GosplOutput2/h5/', 8, scalarAttribute='flowAcc')


In [ ]:
Crash Python Here
'''
movieOutputFileName='GosplAnimation.mp4'
framesPerIteration = 8
subdivisions = 8
scalarAttribute = 'elev'
lookAtLonLat = np.array([60, 20])
cameraZoom = 1.4

outputDir = './GosplOutput2/h5/'
outputPath = Path(outputDir)
numOfFiles = len(list(outputPath.glob('gospl.*.p0.h5')))
fileNamePattern = outputDir + 'gospl.{}.p0.h5'



initMesh = createGosplDataMesh(fileNamePattern, 0, subdivisions=subdivisions)

plotter = pv.Plotter()
plotter.add_mesh(initMesh, scalars=scalarAttribute)
plotter.camera_position = 'yz'
plotter.camera.zoom(cameraZoom)
plotter.camera.azimuth = 180 + lookAtLonLat[0]
plotter.camera.elevation = lookAtLonLat[1]
plotter.show(auto_close=False, window_size=[800, 608])
plotter.open_movie(movieOutputFileName)
for i in range(framesPerIteration):
    plotter.write_frame()


for i in range(numOfFiles-1):
    newMesh = createGosplDataMesh(fileNamePattern, i+1, subdivisions=subdivisions)
    plotter.update_coordinates(newMesh.points, mesh=initMesh)
    plotter.update_scalars(newMesh[scalarAttribute], render=False, mesh=initMesh)
    for i in range(framesPerIteration):
        plotter.write_frame()

plotter.close()
'''

In [ ]:
topologyFileDir = outputDir + 'topology.p0.h5'

#The topology file contains coordinates of a smooth earth and the cells of it's mesh
def readTopologyFile(fileDir):
    with h5py.File(fileDir, "r") as f:
        cells = np.array(f['cells'])
        coords = np.array(f['coords'])
    return coords, cells

In [ ]:


iteration = 14
outputFileNamesPattern = 'GosplOutput2/h5/gospl.{}.p0.h5'

fileDir = outputFileNamesPattern.format(iteration)
with h5py.File(fileDir, "r") as f:
    elev = np.array(f['elev'])
    erodep = np.array(f['erodep'])
    fillAcc = np.array(f['fillAcc'])
    flowAcc = np.array(f['flowAcc'])
    hdisp = np.array(f['hdisp'])
    rain = np.array(f['rain'])
    sedLoad = np.array(f['sedLoad'])
    print(f.keys())
       
topologyFileDir = 'GosplOutput2/h5/topology.p0.h5'
with h5py.File(topologyFileDir, "r") as f:
    cells = np.array(f['cells'])
    coords = np.array(f['coords'])

vertices, cells, heights = earthToIcosphere(earth, subdivisions=8)



print(coords.shape)

outputMesh = createMeshFromNPZdata(vertices, cells, elev, heightAmplification=30)

earthFaces = earthMesh.faces
#outputMesh = pv.PolyData(coords)

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(outputMesh, scalars=rain)
plotter.show()